<a href="https://colab.research.google.com/github/DRSLima/sentiment-analysis-twitter/blob/master/Text%20Classification%20-%20Spanish%20-%20NLP_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

df_train = pd.read_csv('https://raw.githubusercontent.com/jpvmm/FASAM_NLP/master/train_sample.csv')
df_test = pd.read_csv('https://raw.githubusercontent.com/jpvmm/FASAM_NLP/master/test_sample.csv')

In [0]:
#Preparando os dados

from sklearn.model_selection import train_test_split

df_port = df_train[df_train['language']=='portuguese']
df_spani = df_train[df_train['language']=='spanish']

y_por_classes=  pd.get_dummies(df_port['category']).columns
y_por = pd.get_dummies(df_port['category']).values #one_hot_encoding
x_por = df_port['title']

y_sp_classes=  pd.get_dummies(df_spani['category']).columns
y_sp = pd.get_dummies(df_spani['category']).values
x_sp = df_spani['title']

In [5]:
y_por.shape

(17581, 1484)

# **Começando os trabalhos**


---
**Pergunta**: Como podemos trabalhar com duas linguages diferentes? Qual estratégia poderia ser adotada (utilizando Deep Learning)?

## **Primeiro Passo: Download de Embeddings**

Precisamos fazer o download de embeddings tanto em português quanto em espanhol.


In [6]:
#!wget http://143.107.183.175:22980/download.php?file=embeddings/glove/glove_s300.zip #portugues
#!wget https://zenodo.org/record/3255001/files/embeddings-new_large-general_3B_fasttext.vec #espanhol

--2020-01-25 16:12:02--  http://143.107.183.175:22980/download.php?file=embeddings/glove/glove_s300.zip
Connecting to 143.107.183.175:22980... connected.
HTTP request sent, awaiting response... 200 OK
Length: 993190150 (947M) [application/octet-stream]
Saving to: ‘download.php?file=embeddings%2Fglove%2Fglove_s300.zip’

download.php?file=e 100%[===================>] 947.18M  1.13MB/s    in 12m 48s 

2020-01-25 16:24:50 (1.23 MB/s) - ‘download.php?file=embeddings%2Fglove%2Fglove_s300.zip’ saved [993190150/993190150]

--2020-01-25 16:24:51--  https://zenodo.org/record/3255001/files/embeddings-new_large-general_3B_fasttext.vec
Resolving zenodo.org (zenodo.org)... 188.184.95.95
Connecting to zenodo.org (zenodo.org)|188.184.95.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3801508151 (3.5G) [application/octet-stream]
Saving to: ‘embeddings-new_large-general_3B_fasttext.vec’

embeddings-new_larg 100%[===================>]   3.54G  9.80MB/s    in 5m 18s  

2020-01

In [7]:
!mkdir word_embeddings
!unzip download.php\?file\=embeddings%2Fglove%2Fglove_s300.zip

Archive:  download.php?file=embeddings%2Fglove%2Fglove_s300.zip
  inflating: glove_s300.txt          


In [9]:
!mkdir word_embeddings/portugues/
!mkdir word_embeddings/espanhol/

!mv glove_s300.txt word_embeddings/portugues/
!mv embeddings-new_large-general_3B_fasttext.vec word_embeddings/espanhol/

mkdir: cannot create directory ‘word_embeddings/portugues/’: File exists
mkdir: cannot create directory ‘word_embeddings/espanhol/’: File exists
mv: cannot stat 'glove_s300.txt': No such file or directory
mv: cannot stat 'embeddings-new_large-general_3B_fasttext.vec': No such file or directory


In [10]:
# Como o Glove funciona
!head -10 word_embeddings/portugues/glove_s300.txt

929605 300
, 0.112817 0.292620 -0.570195 -0.315653 0.427905 0.034630 0.485927 -0.812266 -0.126687 0.095024 0.346663 -0.324229 -0.361713 0.509577 -0.091133 -0.070473 -0.120127 0.064262 0.343332 0.001556 0.507045 0.394008 -0.437269 0.321949 -0.050355 -0.039882 0.232824 0.225846 0.271784 0.577218 -0.473388 0.401111 0.169473 0.399034 0.127944 0.459724 -0.593398 -0.361026 -0.546738 0.330468 0.233743 0.058388 -0.724790 -0.234829 0.530259 -0.191729 -0.071651 -0.098692 0.145503 -0.346772 -0.033848 0.415718 -0.120032 0.547298 -0.176962 0.058131 -0.248682 0.204490 -0.081491 -0.343670 0.204956 0.254997 0.097111 -0.007818 -0.000967 -0.400638 0.299150 0.296563 0.199632 0.085525 -0.146504 0.097289 0.797291 0.251657 0.438869 -0.096783 -0.315914 -0.216725 0.044398 0.145322 0.567366 -0.190756 -0.076296 0.007373 0.277589 0.327042 -0.191372 -0.170734 -0.246801 -0.851887 -0.179010 -0.112352 -0.165072 0.026478 0.483061 -0.171842 -0.232364 -0.213415 0.349774 -0.220727 0.471147 0.212268 0.827198 0.307026 4.9

## **Segundo Passo: Modelagem dos dados**:

Para utilizarmos um embedding ja treinado com Keras, precisamos modelar os dados para "caber" na camada.

### **Tokenização**



1.   Transformar palavras em números
2.   Tranformar sequências de textos em vetores



In [11]:
#Limpeza de dados

import re
import string
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords

def limpaTexto(texto):

  tokens = texto.split()

  #Regex para filtro de caracteres
  re_puc = re.compile('[%s]' % re.escape(string.punctuation))
  #Remoção de pontuação
  tokens = [re_puc.sub('', w ) for w in tokens]
  #Remoção de tokens não alfabéticos
  tokens = [word for word in tokens if word.isalpha()]
  #Remoção de stopwords
  stop_words = set(stopwords.words('portuguese'))

  tokens = [w for w in tokens if not w in stop_words]
  tokens = [word for word in tokens if len(word) > 1]
  tokens = ' '.join(tokens)

  return tokens

x_por = x_por.apply(limpaTexto)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
# Modelando os dados - Português

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_por)

vocab_size = len(tokenizer.word_index) + 1 #tamanho do vocabulario do dataset
print('O tamanho do vocabulario é:', vocab_size)

Using TensorFlow backend.


O tamanho do vocabulario é: 18951


In [13]:
# Visualizando a tokenização
tokenizer.index_word

{1: 'de',
 2: 'kit',
 3: 'com',
 4: 'para',
 5: 'original',
 6: 'em',
 7: 'preto',
 8: 'do',
 9: 'porta',
 10: 'caixa',
 11: 'branco',
 12: 'peças',
 13: 'frete',
 14: 'led',
 15: 'mesa',
 16: 'unidades',
 17: 'inox',
 18: 'da',
 19: 'azul',
 20: 'mini',
 21: 'bateria',
 22: 'infantil',
 23: 'cm',
 24: 'ar',
 25: 'capa',
 26: 'cabo',
 27: 'sem',
 28: 'jogo',
 29: 'motor',
 30: 'digital',
 31: 'vidro',
 32: 'promoção',
 33: 'novo',
 34: 'profissional',
 35: 'preta',
 36: 'grátis',
 37: 'suporte',
 38: 'usb',
 39: 'par',
 40: 'madeira',
 41: 'aço',
 42: 'pro',
 43: 'rosa',
 44: 'freio',
 45: 'honda',
 46: 'tv',
 47: 'conjunto',
 48: 'tampa',
 49: 'notebook',
 50: 'embreagem',
 51: 'super',
 52: 'parede',
 53: 'controle',
 54: 'bomba',
 55: 'litros',
 56: 'gol',
 57: 'completo',
 58: 'bolsa',
 59: 'placa',
 60: 'brinde',
 61: 'entrega',
 62: 'filtro',
 63: 'elétrica',
 64: 'bivolt',
 65: 'roda',
 66: 'moto',
 67: 'protetor',
 68: 'fio',
 69: 'bebê',
 70: 'vermelho',
 71: 'branca',
 72: 'c

In [19]:
encoded = tokenizer.texts_to_sequences(x_por)

for i in range(5):
  print(x_por.values[i], '| Posicao na lista: ---->', encoded[i])

Microscópio Biológico Binocular Meopta Profissional | Posicao na lista: ----> [1992, 5830, 5831, 8511, 34]
Nissan Versa | Posicao na lista: ----> [631, 1993]
Mini Pci Wireless Bluetooth Samsung | Posicao na lista: ----> [20, 2484, 579, 176, 95]
Fim De Curso Portão Acabamento Aluminio | Posicao na lista: ----> [8512, 1, 2485, 795, 659, 285]
Toalha Picnic Pic Nic Xadrez Oxford Cores Diversas | Posicao na lista: ----> [390, 8513, 5832, 8514, 1806, 1249, 73, 1994]


In [20]:
#Precisamos descobrir o tamanho máximo de uma frase para fazer o padding
max_len = max([len(i) for i in encoded])
print('O tamanho máximo de sequencia de texto é:' ,max_len, '\n')

padded = pad_sequences(encoded, maxlen=max_len, padding='post')

for i in range(5):
  print(x_por.values[i], '| Posicao na lista: ---->', padded[i])

O tamanho máximo de sequencia de texto é: 19 

Microscópio Biológico Binocular Meopta Profissional | Posicao na lista: ----> [1992 5830 5831 8511   34    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]
Nissan Versa | Posicao na lista: ----> [ 631 1993    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]
Mini Pci Wireless Bluetooth Samsung | Posicao na lista: ----> [  20 2484  579  176   95    0    0    0    0    0    0    0    0    0
    0    0    0    0    0]
Fim De Curso Portão Acabamento Aluminio | Posicao na lista: ----> [8512    1 2485  795  659  285    0    0    0    0    0    0    0    0
    0    0    0    0    0]
Toalha Picnic Pic Nic Xadrez Oxford Cores Diversas | Posicao na lista: ----> [ 390 8513 5832 8514 1806 1249   73 1994    0    0    0    0    0    0
    0    0    0    0    0]


### **Carregando Embeddings**



1.   Carregar o GloVe embedding na memória no formato de um dicionário.
2.   Criar uma matriz com cada palavra do vocabulário sendo um vetor de 300 dimensões.

---



In [21]:
embeddings_index = {}

with open('word_embeddings/portugues/glove_s300.txt',encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
print('Foram carregados %s vetores(palavras).' % len(embeddings_index))


Foram carregados 929606 vetores.


In [0]:
embed_size = 300
word_index = tokenizer.word_index
embedding_matrix = np.zeros((vocab_size, embed_size))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [26]:
embedding_matrix[i]

array([-0.010011  , -0.33720401,  0.012428  ,  0.04557   , -0.29369801,
        0.144728  ,  0.56933999,  0.089392  ,  0.15305901,  0.00677   ,
       -0.053562  ,  0.050755  ,  0.424952  , -0.288122  , -0.060534  ,
        0.13498899, -0.012465  , -0.135647  , -0.039961  , -0.191668  ,
       -0.120137  , -0.059094  ,  0.166318  ,  0.47467801,  0.014793  ,
        0.40037   ,  0.051224  ,  0.220385  , -0.31659099, -0.186528  ,
        0.30420801,  0.04599   , -0.349978  ,  0.40175501,  0.007315  ,
        0.099524  , -0.14110801,  0.30479699,  0.310902  , -0.28626001,
        0.112888  ,  0.406064  ,  0.37139699,  0.16442101, -0.126661  ,
        0.309091  , -0.302257  ,  0.23652101, -0.098031  ,  0.105282  ,
       -0.066551  ,  0.077291  , -0.131998  ,  0.39168301,  0.120066  ,
        0.093604  ,  0.083696  , -0.3775    ,  0.035408  ,  0.27917901,
       -0.075074  , -0.17250399, -0.015522  ,  0.062459  , -0.21824899,
        0.322047  ,  0.14771999,  0.218848  , -0.231113  ,  0.01

## **Terceiro Passo: Construção do modelo**

In [45]:
from keras.layers import Dense, Input
from keras.layers import Flatten, LSTM
from keras.layers import Embedding
from keras.models import Model
from keras.callbacks import *
from keras.optimizers import *


inputs1 = Input(shape=(max_len,))
embedding = Embedding(vocab_size, embed_size, weights=[embedding_matrix], trainable=False)(inputs1)
x = LSTM(128)(embedding)
#x1 = LSTM(32)(x)
x1 = Dense(128, activation='relu')(x)
preds = Dense(len(y_por_classes), activation='softmax')(x1)

modelo = Model(inputs=inputs1, outputs=preds)

modelo.summary()

modelo.compile(loss='categorical_crossentropy',
               optimizer='adam',
               metrics=['accuracy'])

weights_filepath='./pesos.h5'

callbacks = [ModelCheckpoint(weights_filepath, monitor='val_loss', mode='min',
                             verbose=1, save_best_only=True),
             EarlyStopping(monitor='val_loss', mode='min', patience=10, verbose=1)]

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 19)                0         
_________________________________________________________________
embedding_11 (Embedding)     (None, 19, 300)           5685300   
_________________________________________________________________
lstm_11 (LSTM)               (None, 128)               219648    
_________________________________________________________________
dense_20 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_21 (Dense)             (None, 1484)              191436    
Total params: 6,112,896
Trainable params: 427,596
Non-trainable params: 5,685,300
_________________________________________________________________


## **Modelagem de dados para treinamento**

In [0]:
x_train_por, x_test_por, y_train_por, y_test_por = train_test_split(padded, y_por, test_size=0.1, random_state=42)

In [47]:
modelo.fit(x_train_por, y_train_por, epochs=40, validation_data=[x_test_por, y_test_por], batch_size=32)

Train on 15822 samples, validate on 1759 samples
Epoch 1/40
15822/15822 [==============================] - 18s 1ms/step - loss: 6.9896 - acc: 0.0030 - val_loss: 6.6744 - val_acc: 0.0040
Epoch 2/40
15822/15822 [==============================] - 16s 1ms/step - loss: 6.2731 - acc: 0.0118 - val_loss: 6.2521 - val_acc: 0.0108
Epoch 3/40
15822/15822 [==============================] - 16s 1ms/step - loss: 5.6894 - acc: 0.0267 - val_loss: 5.8199 - val_acc: 0.0290
Epoch 4/40
15822/15822 [==============================] - 16s 1ms/step - loss: 5.0628 - acc: 0.0650 - val_loss: 5.3359 - val_acc: 0.0733
Epoch 5/40
15822/15822 [==============================] - 16s 1ms/step - loss: 4.2989 - acc: 0.1516 - val_loss: 4.8377 - val_acc: 0.1620
Epoch 6/40
15822/15822 [==============================] - 16s 1ms/step - loss: 3.5014 - acc: 0.2699 - val_loss: 4.3638 - val_acc: 0.2695
Epoch 7/40
15822/15822 [==============================] - 16s 1ms/step - loss: 2.7968 - acc: 0.3862 - val_loss: 4.2156 - val_acc:

In [0]:
preds = modelo.predict(x_test_por)

preds = preds.argmax(axis=1)

preds

In [0]:
from sklearn.metrics import classification_report, accuracy_score

accuracy_score(preds, y_test_por.argmax(1))

# **Desafio**

---


1 - Desenvolva o modelo para espnahol;


2 - Tente melhorar ambos os modelos